In [ ]:
%%HTML
<link rel="stylesheet" type="text/css" href="custom.css">

#### What’s the uncertainty on your ML prediction?

In [ ]:
from IPython.display import Image
Image(filename='img/uncertainty.jpeg', )

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("../errortools/")
import errortools
import scipy.stats
import pandas as pd
import sklearn.preprocessing
import random

import matplotlib

SMALL_SIZE = 20
MEDIUM_SIZE = 24
BIGGER_SIZE = 28

matplotlib.rc('font', size=MEDIUM_SIZE)         # controls default text sizes
matplotlib.rc('axes', titlesize=MEDIUM_SIZE)    # fontsize of the axes title
matplotlib.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
matplotlib.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
matplotlib.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
matplotlib.rc('legend', fontsize=SMALL_SIZE)   # legend fontsize
matplotlib.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

matplotlib.rcParams['font.family'] = 'serif'

from IPython.display import Markdown, display
def Print(string):
    display(Markdown(string))
    
np.random.seed(42)
np.set_printoptions(precision=1)

_Uncertainty/error_ = __how far__ we could be off in our prediction

Roughly two sources of uncertainty  

- Model, assumptions, features, processing, ...  

Sometimes called _systematic uncertainties_

This is not about those

- Limited training data  

Sometimes called _statistical uncertainties_

This is about those

# An illustrative example

We create a dataset according to a perfectly known sigmoid probability distribution  

In [ ]:
slope = 4 
bias  = 0

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 5))
x = np.linspace(-1, 1, 101)
p = scipy.stats.logistic.cdf(slope * x + bias)
ax.plot(x, p, '-', color='black', alpha=1)
ax.set_xlabel("some variable X")
ax.set_ylabel("probability")
ax.grid()
ax.set_ylim((0,1));

In [ ]:
n_traindata = 100
slope = 4 
bias  = 0

X = np.random.uniform(low=-1, high=1, size=n_traindata)
p = scipy.stats.logistic.cdf(X * slope + bias)
y = (p > np.random.uniform(size=n_traindata)).astype(int)

In [ ]:
H, e = np.histogram(X, bins=10, range=(-1,1))
h, e = np.histogram(X[y==1], bins=10, range=(-1,1))
r = h/(H+1e-12)

fig, ax = plt.subplots(1, 1, figsize=(20, 5))
x = np.linspace(-1, 1, 101)
p = scipy.stats.logistic.cdf(slope * x + bias)
ax.plot(X[y==0], y[y==0], 'o', color='red', markersize=10, label="0s")
ax.plot(X[y==1], y[y==1], 'X', color='green', markersize=10, label="1s")
ax.plot(x, p, '--', color='black', alpha=0.8)
#ax.bar((e[:-1]+e[1:])/2., r, e[1]-e[0], color="orange", alpha=0.5, label="fraction of 1s")
ax.set_xlabel("some variable X")
ax.set_ylabel("probability")
ax.grid()
ax.legend()
ax.set_ylim((-0.1,1.1));

We fit a logistic regression to the dataset

In [ ]:
model = errortools.LogisticRegression()
model.fit(X, y)
print(model.parameters)

In [ ]:
H, e = np.histogram(X, bins=10, range=(-1,1))
h, e = np.histogram(X[y==1], bins=10, range=(-1,1))
r = h/(H+1e-12)

fig, ax = plt.subplots(1, 1, figsize=(20, 5))
x = np.linspace(-1, 1, 101)
p = scipy.stats.logistic.cdf(slope * x + bias)
f = model.predict(x)
ax.plot(x, p, '--', color='black', alpha=1, label="true probability: slope {:.0f} bias {:.0f}".format(slope, bias))
#ax.bar((e[:-1]+e[1:])/2., r, e[1]-e[0], color="orange", alpha=0.1)
ax.plot(x, f, '-', color='red', alpha=1, label="model: slope {:.1f} bias {:.1f}".format(model.parameters[0], model.parameters[1]))
ax.set_xlabel("some variable X")
ax.set_ylabel("probability")
ax.grid()
ax.legend()
ax.set_ylim((0,1));

Our model does not get back the exact slope and bias that we put in  

The reason is the dataset  

If we had a different dataset, we would get different values

Let's illustrate this  
We repeat the example many times  

In [ ]:
n_datasets = 1000
Xs = np.random.uniform(low=-1, high=1, size=(n_datasets, n_traindata))
ps = scipy.stats.logistic.cdf(Xs * slope + bias)
ys = (ps > np.random.uniform(size=Xs.shape)).astype(int)

In [ ]:
models = []
for i in range(n_datasets):
    m = errortools.LogisticRegression()
    m.fit(Xs[i], ys[i])
    models.append(m)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,2))
ax[0].hist([m.parameters[0] for m in models], bins=40, range=(2,7), color='orange', alpha=0.5, label="models")
ax[0].plot((slope, slope), ax[0].get_ylim(), color='black', label="True value")
ax[0].grid()
ax[0].set_xlabel("Slope")
ax[0].set_ylabel("Count")
ax[0].legend()

ax[1].hist([m.parameters[1] for m in models], bins=40, range=(-1,1), color='orange', alpha=0.5, label="models")
ax[1].plot((bias, bias), ax[1].get_ylim(), color='black', label="True value")
ax[1].grid()
ax[1].set_xlabel("Bias")
ax[1].legend()

fig, ax = plt.subplots(1, 1, figsize=(15,3))
p = scipy.stats.logistic.cdf(slope * x + bias)
ax.plot(x, models[0].predict(x), '-', color='orange', alpha=0.25, label="models")
for m in models[1:]:
    ax.plot(x, m.predict(x), '-', color='orange', alpha=0.1)
ax.plot(x, p, '--', color='black', label="Real probability")
ax.set_ylim((0,1))
ax.set_xlabel("Some variable X")
ax.set_ylabel("Probability")
ax.legend();

### We see that

A model's parameters and prediction curves depend on the training data  

* And thereby also the AUC, confusion matrix, recall, precision, etc.  

We thus have an uncertainty on our predictions  

Can we somehow estimate these uncertainties?  


### We see that

A model's parameters and prediction curves depend on the training data  

We thus have an uncertainty on our predictions  

# Estimating uncertainties

Two steps

1. Estimate uncertainties on model parameters

2. Propagate uncertainties to predictions

Let's see how this turns out on a real data set

# Uncertainties in action

Surviving the Titanic

Get information on Titanic passengers

In [ ]:
df = pd.read_csv("http://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv")

Do some data mangling

In [ ]:
df['sex'] = df.Sex.apply(lambda s: 1 if s=='female' else 0)
df['has siblings or spouses aboard'] = (df['Siblings/Spouses Aboard'] > 0).astype(int)
df['has parents or children aboard'] = (df['Parents/Children Aboard'] > 0).astype(int)
df['has family aboard'] = (df['Siblings/Spouses Aboard'] + df['Parents/Children Aboard'] > 0).astype(int)

In [ ]:
df.head(5)

Let's look at some data distributions

In [ ]:
dfsurv = df[df.Survived==1]
dfdied = df[df.Survived==0]
fig, ax = plt.subplots(2, 3, figsize=(20, 12))
ax[0,0].hist(df.Pclass, bins=3, range=(0.5,3.5), color='orange', alpha=0.5, label="all")
ax[0,0].hist(dfsurv.Pclass, bins=3, range=(0.5,3.5), color='green', alpha=0.5, label="survived")
ax[0,0].set_xlabel("Pclass")
ax[0,0].legend()

ax[0,1].hist(df.Age, bins=16, range=(0,80), color='orange', alpha=0.5, label="all")
ax[0,1].hist(dfsurv.Age, bins=16, range=(0,80), color='green', alpha=0.5, label="survived")
ax[0,1].set_xlabel("Age")

ax[0,2].hist(df['Siblings/Spouses Aboard'], bins=9, range=(-0.5,8.5), color='orange', alpha=0.5, label="all")
ax[0,2].hist(dfsurv['Siblings/Spouses Aboard'], bins=9, range=(-0.5,8.5), color='green', alpha=0.5, label="survived")
ax[0,2].set_xlabel("Siblings/Spouses Aboard")

ax[1,0].hist(df['Parents/Children Aboard'], bins=7, range=(-0.5,6.5), color='orange', alpha=0.5, label="all")
ax[1,0].hist(dfsurv['Parents/Children Aboard'], bins=7, range=(-0.5,6.5), color='green', alpha=0.5, label="survived")
ax[1,0].set_xlabel("Parents/Children Aboard")

ax[1,1].hist(df['Fare'], bins=20, range=(0,200), color='orange', alpha=0.5, label="all")
ax[1,1].hist(dfsurv['Fare'], bins=20, range=(0,200), color='green', alpha=0.5, label="survived")
ax[1,1].set_xlabel("Fare")

ax[1,2].hist(df['sex'], bins=2, range=(-0.5,1.5), color='orange', alpha=0.5, label="all")
ax[1,2].hist(dfsurv['sex'], bins=2, range=(-0.5,1.5), color='green', alpha=0.5, label="survived")
ax[1,2].set_xlabel("sex")
ax[1,2].get_xaxis().set_ticks([0,1])
ax[1,2].get_xaxis().set_ticklabels(['male', 'female']);

Let's train a model!

In [ ]:
df_train = df.sample(frac=0.5, replace=False, random_state=42)
df_test  = df.loc[df.index.difference(df_train.index)]

In [ ]:
features = ['Pclass', 'Age', 'Fare', 'Siblings/Spouses Aboard', 
            'Parents/Children Aboard', 'sex']
target = 'Survived'

In [ ]:
X_train = df_train[features].values
y_train = df_train[target].values

In [ ]:
X_test = df_test[features].values
y_test = df_test[target].values

In [ ]:
scaler = sklearn.preprocessing.MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
titanic_model = errortools.LogisticRegression()
titanic_model.fit(X_train_scaled, y_train)

## 1. Estimate uncertainties on model parameters

We minimize a loss function $L_{oss}(p; X, y)$  

__And implicitly maximise a likelihood__ $L(p|X,y)\sim e^{-L_{oss}(p; X, y)}$

The optimal model parameters
* minimize the loss function
* are the most likely parameters for the given dataset 

But surrounding parameter values are still likely  

We make a parabolic approximation of the loss function [<sup>1</sup>](#fn1)

This turns the likelihood into a multivariate Gaussian distribution[<sup>2</sup>](#fn2)


<span style="font-size:20px"> 1. $L_{oss}(p; X, y)\approx L_{oss}(p_0) + \frac{1}{2}(p-p_0)\cdot\frac{\partial^2 L_{oss}}{\partial p^2}\cdot(p-p_0)$<br></span>
<span style="font-size:20px"> 2. $L(p|X,y)\approx e^{-\frac{1}{2}(p-p_0)\cdot\frac{\partial^2 L_{oss}}{\partial p^2}\cdot(p-p_0)}$</span>


Let's investigate the parabolic approximation

In [ ]:
_, fig = errortools.report_loss_versus_approximation(titanic_model,
                                                     features + ['bias']);
fig[1]

The approximation automatically gives us the parameter errors.
They are given by the covariance matrix [<sup>1</sup>](#fn1)

* Diagonal elements give the parameter errors
* Off-diagonal elements give the parameter correlations

<span style="font-size:20px"> 1. $\hat{\Sigma}_{p} = \left[\frac{\partial^2 L_{oss}}{\partial p^2}\right]^{-1}$ </span>


## 2. Propagate uncertainties to predictions

A prediction is a function of input features $X$ and model parameters $p$. The most likely model parameters $p_0$ determine the prediction.  

The uncertainties in the parameters propagate to an uncertainty in the prediction. 

**Linear error propagation**  
   * Linear approximation of the prediction function
       * $f(X|p)\approx f(X|p_0)+\frac{\partial f}{\partial p}(X|p_0)\cdot(p-p_0)$
   * Makes the prediction uncertainty a simple equation
       * $\Delta f \approx  \sqrt{\frac{\partial f}{\partial p}(X|p_0) \cdot \hat{\Sigma}_p \cdot \frac{\partial f}{\partial p}(X|p_0)}$  
       
Fast calculation, but may be inexact    

In [ ]:
lower, upper = titanic_model.prediction_errors(X_test, 
                                               method='linear_error_propagation')

**Sample the Gaussian likelihood**  
* Take random parameters $p$ from $L(p|X,y)$[<sup>1</sup>](#fn1)
* Calculate $E_p\left[ \left(f(X|p)-f(X|p_0)\right)^2 \right]$  

Slower, but more exact with more samples

<span style="font-size:20px">1. $L(p|X,y)\approx e^{-\frac{1}{2}(p-p_0)\cdot\hat{\Sigma}_{p}^{-1}\cdot(p-p_0)}$</span>

In [ ]:
lower, upper = titanic_model.prediction_errors(X_test, 
                                               method='sampling')

Now that we have these error estimates calculated. What kind of insights can we obtain about our model?

Uncertainty over the model parameters

In [ ]:
errortools.report_parameter_error(titanic_model, 
                                  features + ['bias'], 
                                  figsize=(20, 4), 
                                  rotation_x_labels=30);

How about the uncertainty over our test samples?

In [ ]:
errortools.report_error_test_samples(titanic_model, 
                                     X_test_scaled,
                                     y_test,
                                     figsize=(20, 10));

In [ ]:
index = 0

Uncertainties over individual predictions

In [ ]:
errortools.report_error_indivial_pred(titanic_model, 
                                      X_test_scaled[index], 
                                      'Fare', features, 0, 3, 50,
                                      figsize=(20,3))


In [ ]:
for f, v, vs in zip(features, X_test[index], X_test_scaled[index]):
    print(f + " : " + str(v) + " (" + str(round(vs, 3)) + " scaled)")

## 3. What's next?!

- Integrate models trained with sklearn
- Error estimation for neural networks (a start has been made with Pytorch)
- Error estimation for regressors like $\chi^2$ fitter
- Error estimation on different metrics like the confusion matrix